In [1]:
import pandas as pd
from datetime import datetime
import os, sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, JavascriptException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select

# executable_path = r"C:\Users\817932702\mamikos\chromedriver\120\chromedriver.exe"
executable_path = r"D:\proyek\chromedriver\120\chromedriver.exe"
service = Service(executable_path)

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False) 
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://mamikos.com/") 

time.sleep(5)

cari = driver.find_element(By. XPATH, '//*[@id="home"]/div[4]/div/div/div/div[1]')
cari.click()

time.sleep(2)

area = driver.find_element(By. XPATH, '//*[@id="home"]/div[13]/div/div[2]/div[2]/ul/li[2]/a')
area.click()

time.sleep(2)

kota = driver.find_element(By. XPATH, '//*[@id="home"]/div[13]/div/div[2]/div[4]/div/div[25]/details/summary')
kota.click()

time.sleep(2)

kota_2 = driver.find_element(By. XPATH, '//*[@id="home"]/div[13]/div/div[2]/div[4]/div/div[25]/details/div/a[5]')
kota_2.click()

time.sleep(2)

kost = driver.find_element(By. XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[1]')
isi_elemen = kost.text

data_elemen = isi_elemen.split('\n')

# Inisialisasi list untuk menyimpan DataFrames
list_df = []

# Lakukan perulangan untuk nomor 1 hingga 20
for i in range(1, 21):
    # Bangun XPath berdasarkan nomor iterasi
    xpath = '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[' + str(i) + ']'
    
    try:
        # Temukan elemen dengan XPath
        elemen = driver.find_element(By.XPATH, xpath)

        # Dapatkan teks dari elemen
        isi_elemen = elemen.text

        # Pisahkan teks menjadi kolom-kolom yang sesuai
        data_elemen = isi_elemen.split('\n')

        # Buat DataFrame
        df = pd.DataFrame({
            'Jenis': [data_elemen[0]],
            'Status Kamar': [data_elemen[1]] if len(data_elemen) > 1 else None,
            'Nama Kost': [data_elemen[2]] if len(data_elemen) > 1 else None,
            'Lokasi': [data_elemen[3]],
            'Fasilitas': [data_elemen[4]],
            'Rating': [data_elemen[5]],
            'Diskon': [data_elemen[6]] if len(data_elemen) > 1 else None,  # Tambahkan penanganan untuk elemen yang tidak memiliki 'Diskon'
            'Harga': [data_elemen[7]]
        })

        # Tambahkan DataFrame ke dalam list
        list_df.append(df)
    except:
        # Tangani kesalahan jika elemen tidak ditemukan
        print(f"Elemen dengan XPath {xpath} tidak ditemukan.")

# Gabungkan semua DataFrames menjadi satu
final_df = pd.concat(list_df, ignore_index=True)

# Tampilkan DataFrame
print(final_df)

time.sleep(2)

next_button = driver.find_element(By. XPATH, '//*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[2]/div/a')
next_button.click()

Elemen dengan XPath //*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[17] tidak ditemukan.
Elemen dengan XPath //*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[18] tidak ditemukan.
Elemen dengan XPath //*[@id="app"]/div/div[5]/div/div[1]/div/div/div[1]/div[1]/div[20] tidak ditemukan.
          Jenis                                       Status Kamar   
0        Campur                                       Sisa 1 kamar  \
1         Putri                                       Sisa 2 kamar   
2        Campur                                       Sisa 1 kamar   
3        Campur                                       Sisa 2 kamar   
4        Campur                                       Sisa 1 kamar   
5        Campur  Kost Singgahsini RR Mansion Tipe D Pancoran Ja...   
6        Campur                                       Sisa 1 kamar   
7        Campur                                       Sisa 3 kamar   
8        Campur                                       Sisa